# 005 Clustering - main analysis

In [1]:
import pandas as pd
import numpy as np
from numpy import arange
import datetime
import matplotlib.pyplot as plt
import csv
import time
import statsmodels.api as sm
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import AgglomerativeClustering
import scipy.cluster.hierarchy as sch


%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [2]:
# Check package versions
import types 
def imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            yield val.__name__

import pkg_resources
root_packages = [i.split('.', 1)[0] for i in list(imports())] 
for m in pkg_resources.working_set:
    if m.project_name.lower() in root_packages:
        print (m.project_name, m.version)

('statsmodels', '0.9.0')
('scipy', '1.1.0')
('pandas', '0.24.0')
('numpy', '1.15.4')
('matplotlib', '2.2.3')


In [ ]:
def convert_bytes(num):
    """this function will convert bytes to MB, GB, etc"""
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0
        
def file_size(file_path):
    """this function will return the file size"""
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)

In [ ]:
fp = r'../../outputs/nighttime_mode_cell_post_daily.csv'
print (file_size(fp))

# 1. Baseline - Home grid cell
* Load baseline of each user
* Calculate the number of people how stay in a given cell
* Compare to actual population (not applicable here)

In [ ]:
# Load baseline
df = pd.read_csv('../../outputs/nighttime_mode_cell_pre_baseline.csv')
print (len(df))
df.head(2)

In [ ]:
df[df['ad_id']=='9520698F-82FE-4831-BDAF-A2F70092FCB7']

In [ ]:
# Calculating the number of people staying in a given cell
df.columns = ['ad_id', 'cell_id', 'mode_count', 'mode_activity']

df_cell = df[['cell_id', 'ad_id']].groupby('cell_id').count().reset_index()
df_cell.columns = ['cell_id', 'baseline']
df_cell.head(2)

In [ ]:
df_cell.describe()

# 2. Result of pre-hurricane

In [ ]:
# Load daily mode of each user
df_pre = pd.read_csv('../../outputs/nighttime_mode_cell_pre_daily.csv')
print (len(df_pre))
df_pre.head(2)

In [ ]:
print (len(df_pre['ad_id'].unique()))
print (sum(df_pre['count']))

In [ ]:
# Calculating the number of people in a given cell at the daily level
df_pre_cell = df_pre[['cell_id', 'ad_id', 'date_revised']].groupby(['cell_id', 'date_revised']).count().reset_index()
df_pre_cell.head(2)

In [ ]:
# Pivot data (columns are dates)
df_pre_cell_pivot = df_pre_cell.pivot(index='cell_id', columns='date_revised', values='ad_id').reset_index()
df_pre_cell_pivot.head(2)

# 3. Result of post-hurricane

In [ ]:
df_post = pd.read_csv('../../outputs/nighttime_mode_cell_post_daily.csv')
print (len(df_post))
df_post.head(2)

In [ ]:
df_post_cell = df_post[['cell_id', 'ad_id', 'date_revised']].groupby(['cell_id', 'date_revised']).count().reset_index()
df_post_cell.head(2)

In [ ]:
df_post_cell_pivot = df_post_cell.pivot(index='cell_id', columns='date_revised', values='ad_id').reset_index()
df_post_cell_pivot.head(2)

# 4. Merge
* Baseline
* pre-hurricane
* Post-hurricane

In [ ]:
df_total = pd.merge(df_cell, df_pre_cell_pivot, how='outer', on='cell_id')
df_total = pd.merge(df_total, df_post_cell_pivot, how='outer', on='cell_id')

In [ ]:
df_total = df_total.reset_index(drop=True)
df_total = df_total.fillna(0)
df_total.head()

# 5. Calculating average number of people in a given cell before Harvey

In [ ]:
date_list_pre = ['2017-08-01',  '2017-08-02', '2017-08-03', '2017-08-04', '2017-08-05', '2017-08-06', 
                 '2017-08-07', '2017-08-08', '2017-08-09', '2017-08-10', '2017-08-11', '2017-08-12', 
                 '2017-08-13', '2017-08-14', '2017-08-15', '2017-08-16','2017-08-17', 
                       '2017-08-18', '2017-08-19', '2017-08-20']

df_total['avg'] = df_total[date_list_pre].mean(axis=1)
df_total.head(2)

# Load grid cell info about %of residential area

In [ ]:
res = pd.read_csv('../../data/harris_county_grid_pct_res.csv')
res.head(2)

In [ ]:
df_total = pd.merge(df_total, res, on='cell_id', how='left')
df_total.head(2)

In [ ]:
df_total[df_total['pct_res']>0].head(2)

In [ ]:
df_total = df_total[df_total['pct_res'].notnull()]

### Drop cells with insufficient pings (<10)

In [ ]:
df_total = df_total[df_total['avg']>10].reset_index(drop=True)

In [ ]:
df_total_T = df_total[['cell_id', '2017-08-01', '2017-08-02', '2017-08-03', '2017-08-04', '2017-08-05', 
                       '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09', '2017-08-10', '2017-08-11', 
                       '2017-08-12', '2017-08-13', '2017-08-14', '2017-08-15', '2017-08-16', '2017-08-17', 
                       '2017-08-18', '2017-08-19', '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23', 
                       '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29', 
                       '2017-08-30', '2017-08-31', '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04', 
                       '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08', '2017-09-09', '2017-09-10', 
                       '2017-09-11', '2017-09-12', '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16', 
                       '2017-09-17', '2017-09-18', '2017-09-19', '2017-09-20', '2017-09-21', '2017-09-22', 
                       '2017-09-23', '2017-09-24', '2017-09-25', '2017-09-26', '2017-09-27', '2017-09-28', 
                       '2017-09-29']].set_index('cell_id').T

df_total_T.head(2)

# 6. Calculating distance from the average to daily users
* Distance = daily number of users - average number of users before Harvey

In [ ]:
df_total.head()

In [ ]:
df_total_dist = df_total.copy()

date_list_post = ['2017-08-01', '2017-08-02', '2017-08-03', '2017-08-04', '2017-08-05', 
                       '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09', '2017-08-10', '2017-08-11', 
                       '2017-08-12', '2017-08-13', '2017-08-14', '2017-08-15', '2017-08-16', '2017-08-17', 
                       '2017-08-18', '2017-08-19', '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23', 
                       '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29', 
                       '2017-08-30', '2017-08-31', '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04', 
                       '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08', '2017-09-09', '2017-09-10', 
                       '2017-09-11', '2017-09-12', '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16', 
                       '2017-09-17', '2017-09-18', '2017-09-19', '2017-09-20', '2017-09-21', '2017-09-22', 
                       '2017-09-23', '2017-09-24', '2017-09-25', '2017-09-26', '2017-09-27', '2017-09-28', 
                       '2017-09-29']

for col in date_list_post:
    df_total_dist[col] = (df_total_dist[col] - df_total_dist['avg']) / df_total_dist['avg']

df_total_dist = df_total_dist.reset_index(drop=True)
df_total_dist.tail()

In [ ]:
df_total_dist_transposed = df_total_dist[['cell_id', '2017-08-01', '2017-08-02', '2017-08-03', '2017-08-04', '2017-08-05', 
                       '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09', '2017-08-10', '2017-08-11', 
                       '2017-08-12', '2017-08-13', '2017-08-14', '2017-08-15', '2017-08-16', '2017-08-17', 
                       '2017-08-18', '2017-08-19', '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23', 
                       '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29', 
                       '2017-08-30', '2017-08-31', '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04', 
                       '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08', '2017-09-09', '2017-09-10', 
                       '2017-09-11', '2017-09-12', '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16', 
                       '2017-09-17', '2017-09-18', '2017-09-19', '2017-09-20', '2017-09-21', '2017-09-22', 
                       '2017-09-23', '2017-09-24', '2017-09-25', '2017-09-26', '2017-09-27', '2017-09-28', 
                       '2017-09-29']].set_index('cell_id').T
df_total_dist_transposed.index = pd.to_datetime(df_total_dist_transposed.index)
df_total_dist_transposed.head(2)

In [ ]:
print (len(df_total_dist_transposed.columns.tolist()))

# 7. Exploing patterns and extract trend line

In [ ]:
decomp_observed = pd.DataFrame(index=date_list_post)
decomp_trend = pd.DataFrame(index=date_list_post)

for c in df_total_dist_transposed.columns.tolist():
    try:
        decomposition = sm.tsa.seasonal_decompose(df_total_dist_transposed[c], model='additive')
        decomp_observed[c] = decomposition.observed
        decomp_trend[c] = decomposition.trend
        
    except ValueError:
        print c

# 8. Clustering of the change of the number of users staying in a given cell
* Preparing transposed dataframe and array
* Converting to numpy array

In [ ]:
# Timeseires with numpy array
myarray = np.transpose(decomp_trend.dropna().as_matrix())
print (myarray.shape)

In [ ]:
list_ctids = decomp_trend.columns.tolist()
print (len(list_ctids))
print (list_ctids[0])

### K-Means Clustering for change (percentage) values between average users and daily users at the grid cell level

In [ ]:
# K means determine K
distortions = []
K = range(1,20)
for k in K:
    kmeanModel = KMeans(n_clusters=k).fit(myarray)
    kmeanModel.fit(myarray)
    distortions.append(sum(np.min(cdist(myarray, kmeanModel.cluster_centers_, 'euclidean'), axis=1))/myarray.shape[0])
    
plt.plot(K, distortions, 'bx-')
plt.xlabel('k')
plt.ylabel('Distortion')
plt.title('The Elbow Method showing the optimal k')
plt.show()

In [ ]:
# Silhoutte test

sil = []

for i in range(2,20):
    km = KMeans(n_clusters=i)
    y_km = km.fit_predict(myarray)
    sil.append(silhouette_score(myarray, y_km))
    
plt.figure()
plt.plot(range(2,20), sil, marker='o', c='b')
plt.show()

In [ ]:
yymmdd_list = range(54)
yymmdd_list_str = ['2017-08-04', '2017-08-05', 
                       '2017-08-06', '2017-08-07', '2017-08-08', '2017-08-09', '2017-08-10', '2017-08-11', 
                       '2017-08-12', '2017-08-13', '2017-08-14', '2017-08-15', '2017-08-16', '2017-08-17', 
                       '2017-08-18', '2017-08-19', '2017-08-20', '2017-08-21', '2017-08-22', '2017-08-23', 
                       '2017-08-24', '2017-08-25', '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29', 
                       '2017-08-30', '2017-08-31', '2017-09-01', '2017-09-02', '2017-09-03', '2017-09-04', 
                       '2017-09-05', '2017-09-06', '2017-09-07', '2017-09-08', '2017-09-09', '2017-09-10', 
                       '2017-09-11', '2017-09-12', '2017-09-13', '2017-09-14', '2017-09-15', '2017-09-16', 
                       '2017-09-17', '2017-09-18', '2017-09-19', '2017-09-20', '2017-09-21', '2017-09-22', 
                       '2017-09-23', '2017-09-24', '2017-09-25', '2017-09-26']

## Agglomerative Clustering

In [ ]:
# Agglomerative clustering dendrogram
plt.figure(figsize=(10,10))
d = sch.dendrogram(sch.linkage(myarray, method='ward'))

In [ ]:
whole_mean = []
whole_3std1 = []
whole_3std2 = []
for d in range(len(myarray[0])):
    A = []
    for n in range(len(myarray)):
        a = myarray[n][d]
        A.append(a)
    whole_mean.append(sum(A)/float(len(A)))    
    whole_3std1.append(3*np.std(A))
    #whole_3std2.append((sum(A)/float(len(A)))-3*np.std(A))

In [ ]:
model = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')

model.fit(myarray)
labels = model.labels_

labels_N = np.zeros([4,myarray.shape[0]])
for n in range(4):
    for i in range(len(myarray)):
        if labels[i]==n:
            labels_N[n][i]=1
        else:
            labels_N[n][i]=0

meancenter4 = np.zeros([4,myarray.shape[1]])
std4 = np.zeros([4,myarray.shape[1]])

for n in range(len(labels_N)):
    for i in range(len(myarray[0])):
        A=[]
        for j in range(len(myarray)):
            a=myarray[j][i]*labels_N[n][j]
            A.append(a)
        meancenter4[n][i] = sum(A)/sum(labels_N[n])
        std4[n][i] = np.std(np.trim_zeros(A))
        

In [ ]:
print len(A)
print len(labels)

# 9. Calculate Area Under Cure for each grids

In [ ]:
list_cts = []
for i in range(len(myarray)):
    list_cts.append(myarray[i]-whole_mean) # Indivicual CTs' coordinates
            
list_0 = []
for i in range(len(labels_N)):
    list_0.append(meancenter4[i]-whole_mean)

In [ ]:
print (len(list_cts)) # Individual CTs 
print (len(myarray)) # Individual CTs original value
print (len(labels)) # clustering results of CTs
print (len(list_ctids)) # list of CT IDs

In [ ]:
def PolygonArea(corners):
    n = len(corners)
    a = 0.0
    for i in range(n):
        j = (i+1) % n
        a += abs(corners[i][0] * corners[j][1] - corners[j][0] * corners[i][1])
        
    area = a / 2.0
    return area

corners = [(0.0,0.0), (1.0,0.0), (0.0,1.0), (1.0, 1.0)]
PolygonArea(corners)

In [ ]:
# Calculate Area Under Curve for each CTs
x_values = range(55)

list_auc = []
for i in range(len(list_cts)):
    xy = zip(x_values, list_cts[i])
    xy = [(0,0)]+xy
    xy.append((54,0))
    area = PolygonArea(xy)
    list_auc.append(area)

print (len(list_auc))
print (list_auc[:5]) #result

In [ ]:
# Code for area calculation of above/below curve - considering positive/negative values
list_auc_2 = []
for c in range(len(list_cts)):
    area_part = []
    for i in range(len(list_cts[c])):
        if i == len(list_cts[c])-1:
            break
        else:
            a = (list_cts[c][i] + list_cts[c][i+1])*0.5
            area_part.append(a)
    #print len(area_part)
    #print sum(area_part)
    list_auc_2.append(sum(area_part))

print (len(list_auc_2))
print (list_auc_2[:5])

In [ ]:
# Area Under Curve of 4 representatives of clustering result
x_values = range(54)
xy_0 = zip(x_values, list_0[0])
xy_0 = [(0,0)]+xy_0
xy_0.append((54,0))
#print xy_0

xy_1 = zip(x_values, list_0[1])
xy_1 = [(0,0)]+xy_1
xy_1.append((54,0))

xy_2 = zip(x_values, list_0[2])
xy_2 = [(0,0)]+xy_2
xy_2.append((54,0))

xy_3 = zip(x_values, list_0[3])
xy_3 = [(0,0)]+xy_3
xy_3.append((54,0))

# 10. Collect results

In [ ]:
myarray_adjusted = []
for i in range(len(myarray)):
    for n in range(4):
#         if group_Kc3[i]==n:
        if labels[i]==n:
            #plt.plot(yymmddhh_list, myarray[i]-whole_mean, color=colors[n], alpha=0.3, linewidth=linewidths[n])
            myarray_adjusted.append(myarray[i]-whole_mean)

In [ ]:
# Mapping
# Merge dataset
re = pd.DataFrame(myarray_adjusted)
re.index = df_total['cell_id']
re.columns = yymmdd_list_str

re['label'] = labels

re['auc'] = list_auc
re['auc_2'] = list_auc_2

re = re.reset_index()
re.head(2)

In [ ]:
re['label'].value_counts()

In [ ]:
print (re[re['label']==0]['auc_2'].mean())
print (re[re['label']==1]['auc_2'].mean())
print (re[re['label']==2]['auc_2'].mean())
print (re[re['label']==3]['auc_2'].mean())

# 11. Magnitude of impact of each neighborhoods
* Depth of curve
* Height of curve

In [ ]:
re['depth'] = re[['2017-08-22', '2017-08-23','2017-08-24','2017-08-25', '2017-08-26', '2017-08-27', 
                  '2017-08-28', '2017-08-29', 
                 '2017-08-30', '2017-08-31', '2017-09-01', '2017-09-02', '2017-09-03',
                 '2017-09-04', '2017-09-05']].min(axis=1) #decreasing CTs
re['height'] = re[['2017-08-22', '2017-08-23','2017-08-24',
                   '2017-08-25', '2017-08-26', '2017-08-27', '2017-08-28', '2017-08-29', 
                 '2017-08-30', '2017-08-31', '2017-09-01', '2017-09-02', '2017-09-03',
                 '2017-09-04', '2017-09-05']].max(axis=1) #increasing CTs

In [ ]:
re['x'], re['y'] = re['cell_id'].str.split(',',1).str
re.head()

# 12. Export agglomerative clustering result

In [ ]:
# re.to_csv('../../outputs/clustering_result.csv', index=False)